In [102]:
import pandas as pd
import re

In [103]:
start_tag = 'Alert Actions'
phq = 'PHQ'

In [146]:
#BUILD DF of PHQ9 data from patient summary
mrns = []
phq9_info = []
f = open('ptsum_copy.txt')
for line in f:
    if start_tag in line:
        cur_mrn = line.split(' ')[0]
    if phq in line: 
        mrns.append(cur_mrn)
        phq9_info.append(line.strip())
    
df_ptsum = pd.DataFrame(data=[mrns,phq9_info],index = ['mrn','phq9_raw']).T
df_ptsum.mrn = df.mrn.astype(int)
df_ptsum.head()

,mrn,phq9_raw
0,36578565,PHQ9: 5/5/15 (0); 8/9/16 (0)
1,36607620,PHQ9: 6/16/15 (6); 8/13/16 (4)
2,28541019,PHQ9: 3/26/16 (2); 10/22/16 (0)
3,29202785,PHQ9: 3/15/2016 (3)
4,28693398,PHQ9: 11/21/15 (0); 07/30/16 (3)


In [145]:
df_form = pd.read_csv('phq9_form_data.txt')

#keep only rows that have at least one value in phq9
df_form = df_form.dropna(subset=df_form.loc[:,'phq_1':'phq_sum'].columns,how='all')

#mrns to int
df_form['mrn'] = df_form['MedRecNo'].astype(int)
df_form.drop(['MedRecNo','wPatName','todayDate'],axis=1,inplace=True)
df_form.head()

,Date,Time,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,...,phq_total2,phq_total3,phq_sum,form:PHQ9:record_rvw_user,form:PHQ9:record_rvw_date,form:PHQ9:record_amnd_user,form:PHQ9:record_amnd_date,form:PHQ9:record_attest_user,form:PHQ9:record_attest_date,mrn
30,2016-03-22,18:05,Not at all,Several days,Several days,Several days,Several days,Not at all,Several days,Not at all,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,27412410
34,2016-05-03,20:47,Not at all,Not at all,Several days,Not at all,Nearly every day,Not at all,Not at all,Not at all,...,NaN,3.0,4.0,NaN,NaN,gras2wn,2016/05/05 12:12,NaN,NaN,27412410
38,2016-05-31,18:47,Not at all,Not at all,Several days,Several days,Several days,Not at all,Several days,Not at all,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,27412410
46,2016-11-08,19:47,Not at all,Not at all,Not at all,Several days,Several days,Several days,Not at all,Not at all,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,27412410
58,2017-01-27,15:32,Several days,Several days,Several days,Several days,Not at all,Not at all,Several days,Not at all,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,27412410


In [ ]:
#get list of mental health MRNs
mrn_want = pd.read_excel('phq9_mrns_only.xlsx')['mrn']

In [153]:
def save_xls(list_dfs, sheet_names,xls_path):
    writer = pd.ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer,sheet_names[n])
    writer.save()

save_xls([df_ptsum,df_form,mrn_want.to_frame()],['PtSummaryData','PHQ9FormData','MentalHealthMRN'],'phq9_data.xlsx')

In [113]:
df['mrn_mental'] = df.mrn.isin(mrn_want)